In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
from os import path
import zipfile


In [46]:
if not path.exists('./goodreads10k/ratings.csv'):
    with zipfile.ZipFile("./goodreads10k/goodreads10k.zip","r") as zip_ref:
        zip_ref.extractall("./goodreads10k")

In [47]:
import pandas as pd

In [48]:
books = pd.read_csv('./goodreads10k/books.csv')
ratings = pd.read_csv('./goodreads10k/ratings.csv')

In [49]:
ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


#### Create dict with goodreads_book_id : category pairs.

The category of a book is its most frequent category tag. 

In [42]:
from goodreads10k.categories import get_categories
category_tags = [
    #'Art',
    #'Biography'
    #'Business',
    #"Chick-Lit",
    "Children-s",
    #'Christian',
    'Classics',
    #'Comics',
    #'Contemporary',
    #'Cookbooks',
    #'Crime',
    'Fantasy',
    #'Fiction',
    #'Graphic-Novels',
    #'Historical-Fiction',
    #'History',
    'Horror',
    #'Humor-and-Comedy',
    #'Manga',
    'Memoir',
    #'Music',
    'Mystery',
    #'Nonfiction',
    'Paranormal',
    #'Philosophy',
    'Poetry',
    #'Psychology',
    #'Religion',
    'Romance',
    #'Science',
    #'Science-Fiction',
    #'Self-Help',
    #'Suspense'
    #'Spirituality',
    #'Sports',
    'Thriller',
    #'Travel',
    #'Young-Adult'
    ]

book_categories = get_categories(category_tags) 

#### Set color for each category

In [76]:
from bokeh.palettes import Category10, Category20, Accent, Set3 # color palletes

book_category_series = books['goodreads_book_id'].apply(lambda bid: book_categories[bid] if bid in book_categories else 'other')

category_color = { ct.lower(): Category10[10][idx]  for idx, ct in enumerate(category_tags)}

book_category_colors = pd.Series([category_color[i] if i in category_color else Category20[20][15] for i in book_category_series ])

In [77]:
book_category_colors[book_category_series == 'horror']

103     #d62728
167     #d62728
175     #d62728
208     #d62728
209     #d62728
         ...   
9899    #d62728
9922    #d62728
9939    #d62728
9953    #d62728
9964    #d62728
Length: 389, dtype: object

In [78]:
from bokeh.plotting import figure, show, output_notebook, save, output_file
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20b
import pandas as pd

output_notebook()

df_combine = pd.read_csv('./EmbeddingGraph/books_final_df.csv')
df_combine['node_color'] = book_category_colors
#df_combine.loc[df_combine.category == 'romance', ['node_color']] = Category20b[20][3]

source = ColumnDataSource(dict(
    x=df_combine['x_tsne'],
    y=df_combine['y_tsne'],
    title= df_combine['title'],
    fill_color=df_combine['node_color'],
    legend_label=book_category_series
))

title = 'PCA visualization of embeddings'

plot_lda = figure(plot_width=2000, plot_height=2000,
                     title='title', tools="pan,wheel_zoom,box_zoom,reset,hover",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x='x', y='y',source=source,
                alpha=0.99, size=10,
                fill_color='fill_color',
                #fill_color=factor_cmap('legend', palette=Category20b[20], factors=df_combine.category.unique()),
                legend_group='legend_label')

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "Title: @title"}

output_file("test.html")
save(plot_lda)
#show(plot_lda)


Loading BokehJS ...

'C:\\pwr\\ONOS\\P4\\Books-ONOS\\test.html'